# Exploring Advanced Prompt Engineering Patterns

In this notebook you will use ChatGPT and LangChain to learn about:

- Chain of Thought Pattern
- Self-Consistency Pattern
- Least to Most Pattern
- ReAct Pattern

___Created By: Dipanjan (DJ)___

## Install OpenAI and LangChain dependencies


In [ ]:
!pip install langchain==0.3.10
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.30
!pip install ddgs==9.6.0
!pip install beautifulsoup4

INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.9 MB/s eta 0:00:00
  Using cached langchain_text_splitters-0.3.11-py3-none-any.whl.metadata (1.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.3/386.3 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.147
    Uninstalling langsmith-0.1.147:
      Successfully uninstalled langsmith-0.1.147
  Attempting uninstall: langchain-c

## Load OpenAI API Credentials

Here we load it from get password function

## Enter API Tokens

In [ ]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

## Load Necessary Dependencies and ChatGPT LLM

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

## Chain-of-Thought (CoT) Pattern

### Zero Shot CoT

In [ ]:
from IPython.display import display, Markdown

prompt = """Given the following game details,
            How many yards was the shortest valid field goal?
            Think step-by-step to get to your answer.
            Show your step-by-step reasoning and then the answer.

            Coming off their win over the Broncos, the Redskins flew to Cowboys Stadium
            for their Week 11 NFC East rivalry match against the Dallas Cowboys.
            After a scoreless first quarter, Washington would strike in the second quarter
            as kicker Shaun Suisham nailed a 35-yard field goal.
            The Redskins would try to add onto their lead in the third quarter
            with Suisham booting a 31-yard field goal which was blocked.
            However, in the fourth quarter, the Cowboys rallied as quarterback Tony Romo
            completing a 10-yard touchdown pass to wide receiver Patrick Crayton.
         """

response = chatgpt.invoke(prompt)
display(Markdown(response.content))

To determine the shortest valid field goal in the game details provided, we need to analyze the information step-by-step.

1. **Understanding Field Goals**: A field goal in American football is a scoring play where the ball is kicked through the opponent's goalposts. The distance of a field goal is measured from the line of scrimmage to the goalposts, plus an additional 10 yards for the end zone.

2. **Identifying Field Goals in the Game**:
   - The first field goal mentioned is a **35-yard field goal** made by Shaun Suisham.
   - The second field goal attempt is a **31-yard field goal**, but it was blocked and therefore does not count as a valid field goal.

3. **Determining Valid Field Goals**: 
   - A valid field goal is one that is successfully kicked through the goalposts. In this case, the only successful field goal mentioned is the 35-yard field goal.

4. **Conclusion**: Since the only successful field goal in the game was the 35-yard field goal, and the blocked attempt does not count, we conclude that the shortest valid field goal is the one that was made.

Thus, the answer is:

**35 yards**.

### Few-Shot CoT

In [ ]:
from IPython.display import display, Markdown

prompt = """Given the following game details,
            How many yards was the shortest valid field goal?
            Think step-by-step to get to your answer similar to the following examples:

            Example 1:
            Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls.
               Each can has 3 tennis balls. How many tennis balls does he have now?

            Reasoning:
              Step 1: Roger started with 5 balls.
              Step 2: 2 cans of 3 tennis balls each is 6 tennis balls in total.
              Step 3: 5 + 6 = 11 tennis balls.

            A: 11 Tennis Balls

            Example 2:
            Q: How many keystrokes are needed to type the numbers from 1 to 500?

            Reasoning:
              Step 1: There are 9 one-digit numbers from 1 to 9.
              Step 2: There are 90 two-digit numbers from 10 to 99.
              Step 3: There are 401 three-digit numbers from 100 to 500.
              Step 4: Adding all of them up, 9 + 90 + 401 = 1392 keystrokes.

            A: 1392 keystrokes


            Q: Coming off their win over the Broncos, the Redskins flew to Cowboys Stadium
            for their Week 11 NFC East rivalry match against the Dallas Cowboys.
            After a scoreless first quarter, Washington would strike in the second quarter
            as kicker Shaun Suisham nailed a 35-yard field goal.
            The Redskins would try to add onto their lead in the third quarter
            with Suisham booting a 31-yard field goal which was blocked.
            However, in the fourth quarter, the Cowboys rallied as quarterback Tony Romo
            completing a 10-yard touchdown pass to wide receiver Patrick Crayton.
         """

response = chatgpt.invoke(prompt)
print(response.content)

To determine the shortest valid field goal from the game details provided, we need to analyze the information step-by-step.

Reasoning:
Step 1: The first field goal mentioned is a 35-yard field goal made by Shaun Suisham in the second quarter.
Step 2: The second field goal attempt is a 31-yard field goal by Suisham in the third quarter, but this attempt was blocked and therefore does not count as a valid field goal.
Step 3: Since the only valid field goal mentioned is the 35-yard field goal, we need to identify if there are any other field goals mentioned in the details. However, there are no other field goals provided in the text.
Step 4: The shortest valid field goal is the only one that was successfully made, which is the 35-yard field goal.

A: 35 yards


## Self-Consistency Pattern

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

cot_prompt_txt = """Given the following problem,
                    Think step-by-step to get to your answer similar to the following example:

                    Example 1:
                    Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls.
                    Each can has 3 tennis balls. How many tennis balls does he have now?

                    Reasoning:
                    Step 1: Roger started with 5 balls.
                    Step 2: 2 cans of 3 tennis balls each is 6 tennis balls in total.
                    Step 3: 5 + 6 = 11 tennis balls.

                    A: 11 Tennis Balls

                    Problem:
                    {problem}
                """

cot_prompt = ChatPromptTemplate.from_template(cot_prompt_txt)
cot_prompt.pretty_print()

================================ Human Message =================================

Given the following problem,
                    Think step-by-step to get to your answer similar to the following example:

                    Example 1:
                    Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls.
                    Each can has 3 tennis balls. How many tennis balls does he have now?

                    Reasoning:
                    Step 1: Roger started with 5 balls.
                    Step 2: 2 cans of 3 tennis balls each is 6 tennis balls in total.
                    Step 3: 5 + 6 = 11 tennis balls.

                    A: 11 Tennis Balls

                    Problem:
                    {problem}
                


In [ ]:
self_con_prompt_txt = """Given the following problem
                         and 3 diverse reasoning paths explored by an AI model
                         analyse these pathways carefully,
                         aggregate, take the majority vote as needed
                         and generate a final single reasoning path along with the answer

                         AI Model Reasoning Path 1:
                         {reasoning_path_1}

                         AI Model Reasoning Path 2:
                         {reasoning_path_2}

                         AI Model Reasoning Path 3:
                         {reasoning_path_3}

                         Problem:
                         {problem}
                    """

self_con_prompt = ChatPromptTemplate.from_template(self_con_prompt_txt)
self_con_prompt.pretty_print()

================================ Human Message =================================

Given the following problem
                         and 3 diverse reasoning paths explored by an AI model
                         analyse these pathways carefully,
                         aggregate, take the majority vote as needed
                         and generate a final single reasoning path along with the answer

                         AI Model Reasoning Path 1:
                         {reasoning_path_1}

                         AI Model Reasoning Path 2:
                         {reasoning_path_2}

                         AI Model Reasoning Path 3:
                         {reasoning_path_3}

                         Problem:
                         {problem}
                    


In [ ]:
# do CoT exploration independently 3 times with these models
gpt1 = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)
gpt2 = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.5)
gpt3 = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.9)

In [ ]:
# do final reasoning and aggregation with this model
chatgpt = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# independent CoT explorations with 3 different LLMs (with different temp settings)
llm_chain1 = cot_prompt | gpt1 | StrOutputParser()
llm_chain2 = cot_prompt | gpt2 | StrOutputParser()
llm_chain3 = cot_prompt | gpt3 | StrOutputParser()

sc_chain = (
                RunnableParallel(problem=RunnablePassthrough(),
                                 reasoning_path_1=llm_chain1,
                                 reasoning_path_2=llm_chain2,
                                 reasoning_path_3=llm_chain3)
                        |
                self_con_prompt
                        |
                    chatgpt
            )

In [ ]:
question = """Q: Coming off their win over the Broncos, the Redskins flew to Cowboys Stadium
            for their Week 11 NFC East rivalry match against the Dallas Cowboys.
            After a scoreless first quarter, Washington would strike in the second quarter
            as kicker Shaun Suisham nailed a 35-yard field goal.
            The Redskins would try to add onto their lead in the third quarter
            with Suisham booting a 31-yard field goal which was blocked.
            However, in the fourth quarter, the Cowboys rallied as quarterback Tony Romo
            completing a 10-yard touchdown pass to wide receiver Patrick Crayton.
            How many yards was the shortest valid field goal?
           """

In [ ]:
response = sc_chain.invoke({'problem': question})
display(Markdown(response.content))

To generate a final reasoning path based on the three provided AI model reasoning paths, we will analyze the key points made in each path and aggregate them to form a cohesive conclusion.

### Aggregated Reasoning Path:

1. **Identification of Field Goals**: The problem states that Shaun Suisham made a successful 35-yard field goal in the second quarter. This is confirmed by all three reasoning paths.

2. **Blocked Field Goal Attempt**: The problem also mentions a 31-yard field goal attempt by Suisham in the third quarter, which was blocked. All three paths correctly identify that a blocked field goal does not count as a valid field goal.

3. **Determining Valid Field Goals**: The only valid field goal mentioned in the problem is the 35-yard field goal. The reasoning across all paths consistently concludes that there are no other valid field goals to consider.

4. **Conclusion on Shortest Valid Field Goal**: Since the only successful field goal is the 35-yard one, it is also the shortest valid field goal. All paths arrive at the same conclusion regarding the shortest valid field goal.

### Final Reasoning Path:
- The problem describes a successful 35-yard field goal made by Shaun Suisham in the second quarter. It also mentions a 31-yard field goal attempt in the third quarter that was blocked, which does not count as valid. Therefore, the only valid field goal is the 35-yard one. Since there are no other valid field goals mentioned, the shortest valid field goal is 35 yards.

### Final Answer:
**35 yards**

## Least to Most Prompting

In [ ]:
PROMPT = """You are a strong reasoning agent.
            Respond to user question with concise and helpful information.
            Follow the format as mentioned in the example,
            break down the problem first into sub problems,
            answer each sub problem sequentially, analyze it
            and then get to the final answer.
            Do not jump directly to the answer

            Example:

            Q: Against Tim Tebow and the Broncos, the two teams would be in a 0-0 deadlock
            in the first half, though the Broncos would nearly score in the second quarter
            on a 28-yard field goal, which would then be blocked by Julius Peppers.
            The Bears would then score 10 points on Marion Barber's 9-yard touchdown run,
            and Robbie Gould's team record-breaking 57-yard field goal, but Tebow's touchdown pass
            to Demaryius Thomas and Matt Prater's 59-yard field goal would tie the game.
            Barber would commit two costly mistakes during the late portion of the game.
            In the fourth quarter, Barber would run out of bounds with 1:55 left,
            and Barber would also fumble in overtime.
            The Broncos would then move downfield and kick a game-winning field goal.
            How many yards was the games longest field goal?

            Response:
            Let's break down this problem into subproblems:
              1. What were the field goals when field goal was?
              2. How many yards was the games longest field goal?

            The answer to subproblems are as follows:
              1. The field goals were 28-yard,  57-yard and  59-yard which are all valid goals
              2. The maximum value out of  59-yard, 28-yard and 57-yard is 59

            A: The final answer is 59 yards.

            Question:
            {query}
        """

query = """Coming off their win over the Broncos, the Redskins flew to Cowboys Stadium
           for their Week 11 NFC East rivalry match against the Dallas Cowboys.
           After a scoreless first quarter, Washington would strike in the second quarter
           as kicker Shaun Suisham nailed a 35-yard field goal.
           The Redskins would try to add onto their lead in the third quarter
           with Suisham booting a 31-yard field goal which was blocked.
           However, in the fourth quarter, the Cowboys rallied as quarterback Tony Romo
           completing a 10-yard touchdown pass to wide receiver Patrick Crayton.
           How many yards was the shortest valid field goal?
        """

prompt = ChatPromptTemplate.from_template(PROMPT)

chain = (prompt
           |
         chatgpt
)

response = chain.invoke({"query": query})
print(response.content)

Let's break down this problem into subproblems:

1. What were the valid field goals attempted in the game?
2. How many yards was the shortest valid field goal?

Now, let's analyze each subproblem:

1. The valid field goals mentioned in the game are:
   - Shaun Suisham's successful 35-yard field goal.
   - A 31-yard field goal attempt by Suisham that was blocked, which does not count as a valid field goal since it was not successful.

2. The only successful field goal is the 35-yard field goal. Since there is only one valid field goal, it is also the shortest.

A: The final answer is 35 yards.


## ReAct Prompting - Agentic AI Flow

In [ ]:
prompt = """Tell me about LangGraph in detail including who made it and key features.
            Do not make up answers, only answer if you are sure"""

# it will either not answer or end up hallucinating the answer
response = chatgpt.invoke(prompt)
display(Markdown(response.content))

As of my last knowledge update in October 2023, there is no widely recognized tool or framework specifically named "LangGraph." It's possible that it could refer to a new project, library, or concept that has emerged after that date, or it may be a niche tool that hasn't gained significant attention in mainstream discussions.

If you have specific details or context about LangGraph, such as its purpose or the domain it operates in, I could help you better understand related concepts or technologies. Otherwise, I recommend checking the latest resources or official documentation for the most accurate and up-to-date information.

In [ ]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_core.tools import tool
from bs4 import BeautifulSoup
import requests
import time
from tqdm import tqdm

@tool
def search_web(query: str) -> list:
    """Search the web for a query."""
    print('Searching web and extracting page info')
    wrapper = DuckDuckGoSearchAPIWrapper(max_results=5)
    results = wrapper.results(query, max_results=5)
    docs = []
    for result in tqdm(results):
        try:
            response = requests.get(result['link'])
            soup = BeautifulSoup(response.content, 'html.parser')
            text_content = soup.get_text(separator="\n", strip=True)
            docs.append(text_content)
        except Exception as e:
            print(f"Error fetching {result['link']}: {e}")
        time.sleep(2)
    return docs

In [ ]:
r = search_web('What is langgraph')

/tmp/ipython-input-2017995428.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  r = search_web('What is langgraph')


Searching web and extracting page info


100%|██████████| 5/5 [00:14<00:00,  2.97s/it]


In [ ]:
print(r[0][:1000])

What is LangGraph? | IBM
What is LangGraph?
AI Agents
Welcome
Caret right
Introduction
Overview
AI agents vs AI assistants
Agentic AI
Agentic AI vs generative AI
Caret right
AI agent development
What is AI agent development?
AgentOps
Caret right
Types of AI agents
Overview
Simple reflex agent
Caret right
Components
Overview
Caret right
Agentic workflows
What are agentic workflows?
Tutorial: Agentic workflows
Communication
Learning
Memory
Perception
Planning
Reasoning
Caret right
Tool calling
What is tool calling?
Tutorial: Ollama tool calling
Tutorial: LM Studio tool calling
Caret right
Architecture
Overview
Caret right
AI agent orchestration
What is agent orchestration?
Tutorial: Agent orchestration
Caret right
Multi-agent systems
What are multi-agent systems?
Tutorial: crewAI multi-agent call analysis
Multi-agent collaboration
Caret right
ReAct
What is ReAct?
Tutorial: LangGraph ReAct agent
Caret right
ReWOO
What is ReWOO?
Tutorial: ReWOO reasoning agent
Caret right
Protocols
Overvie

In [ ]:
chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [search_web]
chatgpt_with_tools = chatgpt.bind_tools(tools)
prompt = "What is langgraph?"
response = chatgpt_with_tools.invoke(prompt)
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UkrFt8qy3ZpV8XBWBWRlBwaa', 'function': {'arguments': '{"query":"langgraph"}', 'name': 'search_web'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 49, 'total_tokens': 64, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_cbf1785567', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--fd16048e-50dd-4c5b-9d73-65828c032025-0', tool_calls=[{'name': 'search_web', 'args': {'query': 'langgraph'}, 'id': 'call_UkrFt8qy3ZpV8XBWBWRlBwaa', 'type': 'tool_call'}], usage_metadata={'input_tokens': 49, 'output_tokens': 15, 'total_tokens': 64, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning':

In [ ]:
response.tool_calls

[{'name': 'search_web',
  'args': {'query': 'langgraph'},
  'id': 'call_UkrFt8qy3ZpV8XBWBWRlBwaa',
  'type': 'tool_call'}]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYS_PROMPT = """You run in a loop of Thought, Action, PAUSE, Observation.
                At the end of the loop, you output an Answer.
                Use Thought to describe your thoughts about the question you have been asked.
                Use Action to run one of the actions available to you - then return PAUSE.
                Observation will be the result of running those actions.
                For the user query below, first break it into multiple sub queries or sub steps if needed.
                The follow the below workflow for each sub query or sub step
                Analyze, validate the relevant results and then compile to generate the final answer

                Use the following workflow format:
                Question: the input task you must solve
                Thought: you should always think about what to do
                Action: the action to take:
                          - should be either to break down the task into sub tasks first if needed
                          - refer to your trained knowledge or call the tool [search_web]
                            if you need to get additional information from the web
                Action Input: the input to the action
                Observation: the result of the action
                ... (this Thought/Action/Action Input/Observation can repeat N times)
                Thought: I now know the final answer
                Final Answer: the final answer to the original input question
             """

prompt_template = ChatPromptTemplate.from_messages(
   [
       ("system", SYS_PROMPT),
       ("human", "Query: {query}"),
       MessagesPlaceholder(variable_name="agent_scratchpad"),
   ]
)

In [ ]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

tools = [search_web]
agent = create_tool_calling_agent(chatgpt, tools, prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
prompt = """Tell me about LangGraph in detail including who made it and key features.
            Do not guess answers, only answer if you are sure"""
response = agent_executor.invoke({"query": prompt})



> Entering new AgentExecutor chain...

Invoking: `search_web` with `{'query': 'LangGraph details, creator, key features'}`


Searching web and extracting page info


100%|██████████| 5/5 [00:11<00:00,  2.36s/it]


['LangGraph\nProducts\nFrameworks\nLangGraph\nLangChain\nPlatforms\nLangSmith\nLangGraph Platform\nResources\nGuides\nBlog\nCustomer Stories\nLangChain Academy\nCommunity\nEvents\nChangelog\nDocs\nPython\nLangGraph\nLangSmith\nLangChain\nJavaScript\nLangGraph\nLangSmith\nLangChain\nCompany\nAbout\nCareers\nPricing\nGet a demo\nSign up\nBalance agent control with agency\nGain control with LangGraph to design agents\nthat reliably handle complex tasks.\nStart building\nIntroduction to LangGraph\nLearn the basics of LangGraph in this LangChain Academy Course. You\'ll learn how to build agents that automate real-world tasks with LangGraph orchestration.\nEnroll for free\nBook enterprise training\nTrusted by companies shaping the future of agents\nSee LangGraph use cases in production\nControllable cognitive architecture for any task\nLangGraph\'s flexible framework supports diverse control flows – single agent, multi-agent, hierarchical, sequential – and robustly handles realistic, complex

In [ ]:
from IPython.display import display, Markdown

display(Markdown(response['output']))

LangGraph is a sophisticated framework designed to enhance the development and deployment of AI agents. It is an extension of the LangChain ecosystem, which is a popular open-source framework for building applications powered by large language models (LLMs). LangGraph introduces a novel approach to creating AI agent runtimes by allowing developers to represent complex workflows as cyclical graphs, providing a more intuitive and flexible way to design agent behaviors.

### Key Features of LangGraph:
1. **Controllable Cognitive Architecture**: LangGraph supports diverse control flows such as single agent, multi-agent, hierarchical, and sequential, allowing for robust handling of complex scenarios.

2. **Human-Agent Collaboration**: It is designed for seamless collaboration between agents and humans, with built-in statefulness that allows agents to write drafts for review and await approval before acting.

3. **Expressive, Customizable Workflows**: LangGraph provides low-level primitives for creating fully customizable agents, supporting diverse control flows using a single framework.

4. **Persistence and Memory**: It includes built-in memory to store conversation histories and maintain context over time, enabling rich, personalized interactions across sessions.

5. **Streaming Support**: LangGraph offers first-class streaming support for better UX design, allowing for real-time display of agent reasoning and actions.

6. **Scalability and Fault Tolerance**: The platform supports fault-tolerant scalability with horizontally-scaling servers, task queues, and built-in persistence, enhancing resilience with intelligent caching and automated retries.

7. **Integrated Developer Experience**: LangGraph Studio simplifies prototyping, debugging, and sharing of agents, with options for 1-click deployment and performance monitoring.

### Creator:
LangGraph is developed by LangChain, a company known for its contributions to the AI and LLM space. The framework is part of their suite of tools designed to build and scale AI workloads effectively.

LangGraph is particularly useful for creating AI agents that can reason, plan, and execute autonomously, making it suitable for applications like customer service bots, research assistants, automated troubleshooting, and content creation pipelines. It is open-source and available under the MIT license, allowing developers to freely use and modify it for their needs.